In [1]:
import pandas as pd

stock_price_by_min = pd.read_csv("../data/raw/TSLA_5_year_by_min.csv")
stock_price_by_min_2018_2020 = pd.read_csv("../data/raw/TSLA_2018_to_2020.csv")
tweets = pd.read_csv("../data/raw/elonmusk_raw.csv")

stock_price_by_min = pd.concat(
    [stock_price_by_min, stock_price_by_min_2018_2020]
).drop_duplicates()

print(stock_price_by_min.info())
print(" ------------------------------------------------------ ")
print(tweets.info())

/var/folders/cw/6fvc9pxs3358jj5gxt73_bth0000gn/T/ipykernel_62611/3835380636.py:5: DtypeWarning: Columns (11,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  tweets = pd.read_csv("../data/raw/elonmusk_raw.csv")


<class 'pandas.core.frame.DataFrame'>
Index: 1602029 entries, 0 to 461420
Data columns (total 10 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   ts_event       1602029 non-null  object 
 1   rtype          1602029 non-null  int64  
 2   publisher_id   1602029 non-null  int64  
 3   instrument_id  1602029 non-null  int64  
 4   open           1602029 non-null  float64
 5   high           1602029 non-null  float64
 6   low            1602029 non-null  float64
 7   close          1602029 non-null  float64
 8   volume         1602029 non-null  int64  
 9   symbol         1602029 non-null  object 
dtypes: float64(4), int64(4), object(2)
memory usage: 134.4+ MB
None
 ------------------------------------------------------ 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55099 entries, 0 to 55098
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  ----

In [9]:
# Lets construct the dataset to predict the stock price in the next X minutes after a
# tweet is posted
import numpy as np

NUM_MINUTES_AHEAD = 5

tweet_wanted_attributes = ["createdAt", "isRetweet", "isReply", "isQuote", "fullText"]
stock_wanted_attributes = ["ts_event", "open", "close"]

# filtering for only original tweets
X = tweets[tweet_wanted_attributes]
X = X[(X["isRetweet"] == False) & (X["isReply"] == False) & (X["isQuote"] == False)]
print(f"Number of original tweets: {len(X)} / {len(tweets)}")

# converting createdAt to datetime
X["createdAt"] = pd.to_datetime(X["createdAt"])

# featurizing day of week and hour of day using sin and cos transformations
X["day_sin"] = X["createdAt"].dt.dayofweek.apply(lambda x: np.sin(2 * np.pi * x / 7))
X["day_cos"] = X["createdAt"].dt.dayofweek.apply(lambda x: np.cos(2 * np.pi * x / 7))
X["hour_sin"] = X["createdAt"].dt.hour.apply(lambda x: np.sin(2 * np.pi * x / 24))
X["hour_cos"] = X["createdAt"].dt.hour.apply(lambda x: np.cos(2 * np.pi * x / 24))

# lets now get the stock prices NUM_MINUTES_AHEAD after each tweet
stock_price_by_min["ts_event"] = pd.to_datetime(stock_price_by_min["ts_event"])
y = stock_price_by_min[stock_wanted_attributes]

# create window start and end times to join on
X["window_start_time"] = X["createdAt"].dt.ceil("min")
X["window_end_time"] = X["window_start_time"] + pd.Timedelta(minutes=NUM_MINUTES_AHEAD)

X = X.sort_values("createdAt").reset_index(drop=True)
y = y.sort_values("ts_event").reset_index(drop=True)

# filter X to only have tweets that have stock price data after them
X = X[y["ts_event"].min() <= X["window_start_time"]]
# filter to weekdays and market hours (data is in UTC)
# this range is a little wider to account for Daylight Savings Time changes
X = X[
    (X["createdAt"].dt.dayofweek < 5)
    & (X["createdAt"].dt.hour >= 13)
    & (X["createdAt"].dt.hour < 21)
]

# join to get stock prices at the start of the window
data = pd.merge_asof(
    X, y, left_on="window_start_time", right_on="ts_event", direction="forward"
)
# join to get stock prices at the end of the window
data = pd.merge_asof(
    data,
    y,
    left_on="window_end_time",
    right_on="ts_event",
    direction="forward",
    suffixes=("_start", "_end"),
)

# calculate the % change and convert to basis points (0.01%) so we dont have to deal
# with small decimals
data["bps_change"] = (
    10000 * (data["close_end"] - data["open_start"]) / data["open_start"]
)

# drop unnecessary columns
data = data.drop(
    columns=[
        "createdAt",
        "isRetweet",
        "isReply",
        "isQuote",
        "ts_event_start",
        "ts_event_end",
        "open_start",
        "open_end",
        "close_start",
        "close_end",
        "window_start_time",
        "window_end_time",
    ]
)

print(data.info())
data.head()

Number of original tweets: 6857 / 55099
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1330 entries, 0 to 1329
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   fullText    1330 non-null   object 
 1   day_sin     1330 non-null   float64
 2   day_cos     1330 non-null   float64
 3   hour_sin    1330 non-null   float64
 4   hour_cos    1330 non-null   float64
 5   bps_change  1330 non-null   float64
dtypes: float64(5), object(1)
memory usage: 62.5+ KB
None


,fullText,day_sin,day_cos,hour_sin,hour_cos,bps_change
0,Oh and uh short burn of the century comin soon...,-0.433884,-0.900969,-0.258819,-0.965926,1.056859
1,Rococo basilisk,0.000000,1.000000,-0.965926,0.258819,44.054334
2,Night at the museum. G has mad skillzsz. Me no...,0.974928,-0.222521,-0.965926,-0.258819,11.294180
3,@NannySoleil @CatherineUllo We sketched concep...,0.974928,-0.222521,-0.965926,-0.258819,19.966722
4,It’s super messed up that a Tesla crash result...,0.000000,1.000000,-0.866025,0.500000,-2.402773


In [ ]:
# Now we need to vectorize the text. Let's use a pre-trained embedding model
from sentence_transformers import SentenceTransformer

# https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# encode the full text to get embeddings - embedding dim is 384
embeddings = model.encode(data["fullText"].tolist(), show_progress_bar=True)
print("Shape of embedding tensor:", embeddings.shape)

Batches:   0%|          | 0/42 [00:00<?, ?it/s]

Shape of embedding tensor: (1330, 384)


In [ ]:
# Let's create a dataframe for the embeddings
embedding_cols = [f"emb_{i}" for i in range(embeddings.shape[1])]

# Add as separate columns (concate horizontally matching on same index)
embedding_df = pd.DataFrame(embeddings, columns=embedding_cols, index=data.index)
data = pd.concat([embedding_df, data], axis=1)

# drop unnecessary fullText column since we have it vectorized now
data = data.drop(columns=["fullText"])

data.head()

,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,...,emb_379,emb_380,emb_381,emb_382,emb_383,day_sin,day_cos,hour_sin,hour_cos,bps_change
0,-0.001008,0.052315,-0.009976,-0.014350,0.043964,-0.011720,-0.076216,-0.042937,-0.015438,-0.032219,...,0.000549,0.016286,-0.039692,0.012335,0.026120,-0.433884,-0.900969,-0.258819,-0.965926,1.056859
1,-0.079920,-0.016484,-0.065066,0.012709,0.020921,-0.029911,-0.025508,0.067164,-0.005198,-0.059973,...,0.048994,-0.007372,-0.064891,0.070081,0.054620,0.000000,1.000000,-0.965926,0.258819,44.054334
2,-0.113694,0.040588,-0.036773,-0.022655,-0.058566,-0.035179,0.050456,-0.056513,-0.025926,-0.040562,...,-0.015642,0.101047,0.040583,-0.122994,-0.003087,0.974928,-0.222521,-0.965926,-0.258819,11.294180
3,-0.079638,-0.041545,-0.010374,-0.044825,0.038043,-0.031383,0.033809,0.011510,0.006959,0.023301,...,0.014632,0.036524,0.021378,-0.023749,0.066412,0.974928,-0.222521,-0.965926,-0.258819,19.966722
4,-0.020206,-0.010676,0.045836,0.060031,0.089319,0.014100,-0.034083,0.032532,-0.035500,0.059870,...,-0.002166,0.012382,0.012327,-0.004536,0.081259,0.000000,1.000000,-0.866025,0.500000,-2.402773


In [16]:
from sklearn.model_selection import train_test_split

# split it into train, val and test sets
X, y = data[data.columns[:-1]], data["bps_change"]

# temporal split - no shuffling. 70-15-15 split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, shuffle=False
)

print(f"Train size: {len(X_train)}, Val size: {len(X_val)}, Test size: {len(X_test)}")

Train size: 931, Val size: 199, Test size: 200


In [ ]:
# Lets finally fit a linear regression model for a baseline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


def fit_linear_model(model, X_train, y_train, X_val, y_val):
    model.fit(X_train, y_train)

    # Get metrics
    R2_train = model.score(X_train, y_train)
    MSE_train = mean_squared_error(y_train, model.predict(X_train))

    R2_val = model.score(X_val, y_val)
    MSE_val = mean_squared_error(y_val, model.predict(X_val))

    print(f"Train R2: {R2_train}, Train MSE: {MSE_train}")
    print(f"Validation R2: {R2_val}, Validation MSE: {MSE_val}")


fit_linear_model(LinearRegression(), X_train_bow, y_train, X_val_bow, y_val)